In [1]:
#Content Based Model by Rating


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics.pairwise import cosine_similarity 


In [3]:
#let's say we have 2 category for curry Fruity and Spicy
#and 4 people. 2 like Fruity and 2 like Spicy
features = ["FruitySet1","FruitySet2","SpicySet1","SpicySet2"]
heidi = [0.8,0.7,-0.1,-0.1]
henry  = [0.7,0.8,-0.2,-0.1]
peter  = [-0.1,-0.1,0.8,0.7]
pedro  = [-0.1,-0.2,0.7,0.7]


classes = ["Frank","Ferdy","Sam","Samuel"]
datas = np.stack([heidi,henry,peter,pedro])
df_data = pd.DataFrame(datas,columns=features,index=classes)
df_data
#below frame shows user by curryse features

,FruitySet1,FruitySet2,SpicySet1,SpicySet2
Frank,0.8,0.7,-0.1,-0.1
Ferdy,0.7,0.8,-0.2,-0.1
Sam,-0.1,-0.1,0.8,0.7
Samuel,-0.1,-0.2,0.7,0.7


In [4]:
#if we get the transpose now it is curryset by use feature
df_curryset_user = df_data.T
df_curryset_user

,Frank,Ferdy,Sam,Samuel
FruitySet1,0.8,0.7,-0.1,-0.1
FruitySet2,0.7,0.8,-0.1,-0.2
SpicySet1,-0.1,-0.2,0.8,0.7
SpicySet2,-0.1,-0.1,0.7,0.7


In [5]:
#now we have a new user(me) ,and I rated only 2 items
new_person_me  = [-0.1,0,0.7,0]
#new_person  = [ FruitySet1 = -0.1, FruitySet2 = 0, SpicySet1 = 0.7, SpicySet2= 0 ]

In [6]:

#generate new dataframe
classes = classes + ["me"]
print("classes",classes)
datas = np.stack([heidi,henry,peter,pedro,new_person_me])

df_data = pd.DataFrame(datas,columns=features,index=classes)
df_data

classes ['Frank', 'Ferdy', 'Sam', 'Samuel', 'me']


,FruitySet1,FruitySet2,SpicySet1,SpicySet2
Frank,0.8,0.7,-0.1,-0.1
Ferdy,0.7,0.8,-0.2,-0.1
Sam,-0.1,-0.1,0.8,0.7
Samuel,-0.1,-0.2,0.7,0.7
me,-0.1,0.0,0.7,0.0


In [7]:
#now here the important,just filter users who gave similar ratings to similar items
similar_likes = df_data.iloc[[2,3]]
similar_likes

,FruitySet1,FruitySet2,SpicySet1,SpicySet2
Sam,-0.1,-0.1,0.8,0.7
Samuel,-0.1,-0.2,0.7,0.7


In [8]:
similar_likes = similar_likes.T
similar_likes

,Sam,Samuel
FruitySet1,-0.1,-0.1
FruitySet2,-0.1,-0.2
SpicySet1,0.8,0.7
SpicySet2,0.7,0.7


In [9]:
#if i get cosine of curryset by curryset ,it is a similarity matrix for currysets
#i can use this as a feature vector
cosine_similarities = cosine_similarity(similar_likes, similar_likes) 
cosine_pd = pd.DataFrame(cosine_similarities,index=features,columns=features)
cosine_pd

,FruitySet1,FruitySet2,SpicySet1,SpicySet2
FruitySet1,1.000000,0.948683,-0.997785,-1.000000
FruitySet2,0.948683,1.000000,-0.925547,-0.948683
SpicySet1,-0.997785,-0.925547,1.000000,0.997785
SpicySet2,-1.000000,-0.948683,0.997785,1.000000


In [10]:
user_id = 4
df_byuser = df_data
cosine_pd
ratings = df_data

In [11]:
df_byuser.iloc[user_id].values

array([-0.1,  0. ,  0.7,  0. ])

In [12]:
user_row = df_byuser.iloc[user_id].values
rated_indeces = np.argwhere(user_row != 0)
not_rated_indeces = np.argwhere(user_row == 0)
print("how many items rated by user",len(rated_indeces))
print("how many items NOT rated by user",len(not_rated_indeces))

how many items rated by user 2
how many items NOT rated by user 2


In [13]:
ratings_by_new_user = user_row[rated_indeces]   
print("ratings by user",ratings_by_new_user)

ratings by user [[-0.1]
 [ 0.7]]


In [14]:
#now get a slice of cosine table with the items this use rated
 
print("how many items rated  positive",len(ratings_by_new_user[ratings_by_new_user>0]))        

#find rated items
rateds = cosine_pd.iloc[rated_indeces.flatten()]

how many items rated  positive 1


In [15]:
rateds

,FruitySet1,FruitySet2,SpicySet1,SpicySet2
FruitySet1,1.000000,0.948683,-0.997785,-1.000000
SpicySet1,-0.997785,-0.925547,1.000000,0.997785


In [16]:
row_with_rating = []
for index,row_values in enumerate(rateds.values):
    weighted_cosine = ratings_by_new_user[index] * row_values
    print("index :",index)
    print("ratings[index] ",ratings_by_new_user[index])
    print("row_values :",row_values)
    print("weighted_cosine :",weighted_cosine)
    row_with_rating.append( weighted_cosine )
row_with_rating
#new_user rated rated FruitySet1  and SpicySet1

#new_user rating for FruitySet1 is 0.1
#we multiply this value FruitySet1 row of cosine table

#new_user rating for SpicySet1 is 0.7
#we multiply this value SpicySet1 row of cosine table
pd.DataFrame(row_with_rating,columns=features)

index : 0
ratings[index]  [-0.1]
row_values : [ 1.          0.9486833  -0.99778516 -1.        ]
weighted_cosine : [-0.1        -0.09486833  0.09977852  0.1       ]
index : 1
ratings[index]  [0.7]
row_values : [-0.99778516 -0.92554696  1.          0.99778516]
weighted_cosine : [-0.69844961 -0.64788287  0.7         0.69844961]


,FruitySet1,FruitySet2,SpicySet1,SpicySet2
0,-0.10000,-0.094868,0.099779,0.10000
1,-0.69845,-0.647883,0.700000,0.69845


In [17]:
#now i have weighted values of currysets by user. so i can sum them column wise
#and this gives me a score of how will user rate this item,by checking our previous ratings
#as you see we predict he will like spicy food but not fruity(sweet) one
#so i can say we captured his likes
print("row_with_rating shape ",np.array(row_with_rating).shape )
totalpoints = np.sum(row_with_rating, axis=0)
print("totalpoints :",totalpoints.shape)
print("totalpoints:",totalpoints)

row_with_rating shape  (2, 4)
totalpoints : (4,)
totalpoints: [-0.79844961 -0.7427512   0.79977852  0.79844961]


In [18]:
df_all=pd.DataFrame(totalpoints,columns=["point"],index=features)
df_all

,point
FruitySet1,-0.798450
FruitySet2,-0.742751
SpicySet1,0.799779
SpicySet2,0.798450
